In [1]:
import fuzzingbook_utils

# Guiging GUI Testing with Mined UI Grammars

__GOAL:__ Mine a single grammar containing UI Elements, UI States, UI Interactions and Statement Coverage. Consume this grammar to efficiently generate tests.

## Roadmap

1. __DONE__ Explore an app using DroidMate _(maybe mutiple times?)_.
2. __ONGOING__ Extract a UI grammar containing only the UI elements which DM interacted with (Kotlin).
3. __DONE__ Use the fuzzingbook's GrammarCoverageFuzzer to consume the Grammar and generate input values (Python).
4. __DONE__ Extend the funzzingbook's grammar coverage tracking to consider both terminals and non-terminals (Python). 
5. _PENDING_ Associate the grammar elements with _covered statements_ (Kotlin).
6. _PENDING_ Consume expanded grammar to generatetests targeted to specific code locations (Kotlin).
7. _DONE_ Generate inputs to cover all grammar elements
8. _PENDING_ Generate inputs to cover all previously covered statements
9. _PENDING_ Generate inputs to cover specific statements

## Grammar definition:

The grammar contains two types of productions _<state>_ and _<transition>_, where:

### State (Non-Terminal <sX\>)

Represents a UI state. It contains all _UI elements_ which can be interacted with on the state, alongside their transitions. 
It is defined as 

```
<sX> ::= w1<t1>, ..., wN<tN>
```

Where:
* wX is a widget
* <tX\> is a transition

### Transition (Non-Terminal <tX\>)

Represents a UI state transition trigger by a UI element interaction. 

By design, is always associated to a UI element, but could be theoretically generalized for system events.
It is defined as

```
  <tX> ::= <s1>, ..., <sN>
```

Where _<sX\>_ is a state. 
A transition contains only UI state productions. 
It also contains _all UI states_ which can be reached by this transition.

### Widget/UI Element (Terminal wX)

A UI element represents a UI element interaction in the app. It is defined as 

```
wX ≡ widgetX.actionY[.dataZ]
```

Where:
* _widgetX_ is the unique UI element ID
* _actionY_ is the type of action performed
* _dataZ_ is the value used in the action, such as text input, if any. This parameter is optional.

This notation can simultaneously encode:
* UI elements
* Different action types on the same UI elements (click, long click, tick, etc, input text)
* Different input payloads (searched value)

### Terminate condition

An exploration can be over at any time. Thus, all _State_ productions can also be $\epsilon$.



# Concrete Example

## Notation

* State: Sequential numbering
* Widget: Sequential numbering (later mapped to UID)
* Action types: Click and Long Click
* Transition: <state.widget>. 
For example `s6.w7.Click` stands for: the widget `w7.Click` in the state `s6`.
This allows the same widget and interaction to have different transitions in different states.


## Grammar

In [2]:
simple_ui_grammar = {
'<empty>'             : 		[''],
'<s1.w0.Click>'       : 		['<empty>', '<s2>'],
'<s1>'                : 		['w0.Click <s1.w0.Click>'],
'<s2.w1.Click>'       : 		['<empty>', '<s3>'],
'<s2.w3.Click>'       : 		['<empty>', '<s4>'],
'<s2>'                : 		['w1.Click <s2.w1.Click>', 'w3.Click <s2.w3.Click>'],
'<s3.w2.Click>'       : 		['<empty>', '<s2>'],
'<s3>'                : 		['w2.Click <s3.w2.Click>'],
'<s4.w4.Click>'       : 		['<empty>', '<s4>'],
'<s4.w5.LongClick>'   : 		['<empty>', '<s5>'],
'<s4>'                : 		['w4.Click <s4.w4.Click>', 'w5.LongClick <s4.w5.LongClick>'],
'<s5.w6.Click>'       : 		['<empty>', '<s6>'],
'<s5>'                : 		['w6.Click <s5.w6.Click>'],
'<s6.w7.Click>'       : 		['<empty>', '<s6>'],
'<s6.w8.Click>'       : 		['<empty>', '<s6>'],
'<s6>'                : 		['w7.Click <s6.w7.Click>', 'w8.Click <s6.w8.Click>'],
'<start>'             : 		['<empty>', '<s1>']
}

# Fuzzing

## Using expansion coverage

With fuzzingbook's default implementation

In [3]:
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

In [18]:
fuzzer = GrammarCoverageFuzzer(simple_ui_grammar, min_nonterminals=1)
max_exp = fuzzer.max_expansion_coverage(max_depth=len(simple_ui_grammar))
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    print(fuzzer.fuzz())

w0.Click 

w0.Click w3.Click w5.LongClick 
w0.Click w1.Click w2.Click 
w0.Click w1.Click 
w0.Click w3.Click 
w0.Click w3.Click w4.Click w5.LongClick w6.Click 
w0.Click w3.Click w4.Click 
w0.Click w1.Click w2.Click w3.Click w5.LongClick w6.Click w8.Click 
w0.Click w3.Click w5.LongClick w6.Click w7.Click 
w0.Click w3.Click w5.LongClick w6.Click w8.Click w7.Click w7.Click w8.Click 


## Using Term Coverage (Terminals and non-Terminals)

Extension of fuzzingbook's implementation.

Greedy algorithm.

In [5]:
import re
from Grammars import START_SYMBOL, RE_NONTERMINAL, nonterminals
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

### Extended Coverage measurement

Number of times each symbol has been seem (necessary as all states can finish with $\epsilon$)

In [6]:
class TerminalCoverageGrammar(GrammarCoverageFuzzer):
    def add_coverage(self, symbol, new_children):
        """Coverage computation: number of times each symbol has been seen"""
        if self.log and symbol not in self.covered_expansions.keys():
            print("Now covered:", symbol)

        if symbol not in self.covered_expansions.keys():
            self.covered_expansions[symbol] = 0

        curr_val = self.covered_expansions[symbol]
        self.covered_expansions[symbol] = curr_val + 1
    def reset_coverage(self):
        self.covered_expansions = dict()

    def expansion_coverage(self):
        return self.covered_expansions.keys()

### Obtain set of existing symbols

Compute all existing symbols (terminals and non terminals) in the grammar:

In [7]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def all_symbols_grammar(self):
        symbols = { START_SYMBOL }

        for symbol in self.grammar.keys():
            for expansion in self.grammar[symbol]:
                for expansion_segment in list(filter(lambda x: x != "", re.split(RE_NONTERMINAL, expansion))):
                    symbols.add(expansion_segment)

        return symbols


### Redefine non-covered elements

Expansions with non-covered elements must use the new data type:

In [8]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def _uncovered_children(self, possible_children):
        # Prefer uncovered expansions
        uncovered_children = []
        index_map = []
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple
                if child not in self.expansion_coverage() and child_list not in uncovered_children:
                    uncovered_children.append(child_list)
                    index_map.append(idx)

        return index_map, uncovered_children

### Determine minimally covered elements

The same UI elements may be available in multiple states. Thus we should prioretize elements which have been seen the least when all elements have already been covered.

In [9]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def _minimally_expanded(self, possible_children):
        symbol_count = dict()
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple

                assert(child in self.covered_expansions.keys())
                symbol_count[child] = self.covered_expansions[child]

        min_count = min(symbol_count.values())

        # Prefer least covered expansions
        uncovered_children = []
        index_map = []
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple
                if symbol_count[child] == min_count and child_list not in uncovered_children:
                    uncovered_children.append(child_list)
                    index_map.append(idx)

        return index_map, uncovered_children

### Add terminals to coverage

Add terminals to the list of covered grammar elements

In [10]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def process_chosen_children(self, chosen_children, expansion):
        """Add terminals to coverage."""
        for expansion_segment in list(filter(lambda x: x != "", re.split(RE_NONTERMINAL, expansion))):
            if expansion_segment not in nonterminals(expansion):
                self.add_coverage(expansion_segment, chosen_children)
        return chosen_children


### Choose node expansion

Update node expansion rules to guide expansion towards least explored elements

In [11]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def choose_node_expansion(self, node, possible_children):
        index_map, uncovered_children = self._uncovered_children(possible_children)

        if len(uncovered_children) == 0:
            # Search for least explored
            index_map, min_covered_children = self._minimally_expanded(possible_children)
            # All expansions covered the same amount of times - use superclass method
            index = self.choose_covered_node_expansion(node, min_covered_children)

            return index_map[index]

        # Select from uncovered nodes
        index = self.choose_uncovered_node_expansion(node, uncovered_children)

        return index_map[index]

### Running the Fuzzer

In [15]:
fuzzer = TerminalCoverageGrammar(simple_ui_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    print(fuzzer.fuzz())

w0.Click 
w0.Click w1.Click w2.Click w3.Click w4.Click w5.LongClick w6.Click w8.Click w7.Click 
